# Prediccion de Default en Prestamos


Para este proyecto utilizaremos un sample de los datos de Lending Club. La idea es predecir si cierto usuario cometera Default basado en informacion que la plataforma recolecta. Esto nos ayudara a mejorar la metodologia/pipeline de prestamo.


In [1]:
import pandas as pd
import numpy as np
import gzip as gz
import json
pd.set_option('display.max_columns', None)

In [2]:
loans=pd.read_csv('https://github.com/sonder-art/fdd_prim_2023/blob/main/codigo/pandas/LoansData_sample.csv.gz?raw=true', compression='gzip') 


/tmp/ipykernel_4200/3179421677.py:1: DtypeWarning: Columns (19) have mixed types. Specify dtype option on import or set low_memory=False.
  loans=pd.read_csv('https://github.com/sonder-art/fdd_prim_2023/blob/main/codigo/pandas/LoansData_sample.csv.gz?raw=true', compression='gzip')


In [3]:
loans.head()

,Unnamed: 0,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,desc,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,fico_range_low,fico_range_high,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,next_pymnt_d,last_credit_pull_d,last_fico_range_high,last_fico_range_low,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,application_type,annual_inc_joint,dti_joint,verification_status_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_act_il,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_bc_dlq,mths_since_recent_inq,mths_since_recent_revol_delinq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,revol_bal_joint,sec_app_fico_range_low,sec_app_fico_range_high,sec_app_earliest_cr_line,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_chargeoff_within_12_mths,sec_app_collections_12_mths_ex_med,sec_app_mths_since_last_major_derog,hardship_flag,hardship_type,hardship_reason,hardship_status,deferral_term,hardship_amount,hardship_start_date,hardship_end_date,payment_plan_start_date,hardship_length,hardship_dpd,hardship_loan_status,orig_projected_additional_accrued_interest,hardship_payoff_balance_amount,hardship_last_payment_amount,disbursement_method,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term
0,0,38098114,NaN,15000.0,15000.0,15000.0,60 months,12.39,336.64,C,C1,MANAGEMENT,10+ years,RENT,78000.0,Source Verified,Dec-2014,Fully Paid,n,NaN,debt_consolidation,Debt consolidation,235xx,VA,12.03,0.0,Aug-1994,750.0,754.0,0.0,NaN,NaN,6.0,0.0,138008.0,29.0,17.0,w,0.00,0.00,17392.37,17392.37,15000.00,2392.37,0.0,0.00,0.0000,Jun-2016,12017.81,NaN,Nov-2017,684.0,680.0,0.0,NaN,1.0,Individual,NaN,NaN,NaN,0.0,0.0,149140.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,184500.0,NaN,NaN,NaN,5.0,29828.0,9525.0,4.7,0.0,0.0,103.0,244.0,1.0,1.0,0.0,47.0,NaN,NaN,NaN,0.0,1.0,4.0,1.0,2.0,8.0,5.0,9.0,4.0,6.0,0.0,0.0,0.0,4.0,100.0,0.0,0.0,0.0,196500.0,149140.0,10000.0,12000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
1,1,36805548,NaN,10400.0,10400.0,10400.0,36 months,6.99,321.08,A,A3,Truck Driver Delivery Personel,8 years,MORTGAGE,58000.0,Not Verified,Dec-2014,Charged Off,n,NaN,credit_card,Credit card refinancing,937xx,CA,14.92,0.0,Sep-1989,710.0,714.0,2.0,42.0,NaN,17.0,0.0,6133.0,31.6,36.0,w,0.00,0.00,6611.69,6611.69,5217.75,872.67,0.0,521.27,93.8286,Aug-2016,321.08,NaN,Feb-2017,564.0,560.0,0.0,59.0,1.0,Individual,NaN,NaN,NaN,0.0,0.0,162110.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,19400.0,NaN,NaN,NaN,7.0,9536.0,7599.0,41.5,0.0,0.0,76.0,290.0,1.0,1.0,1.0,5.0,42.0,1.0,42.0,4.0,6.0,9.0,7.0,18.0,2.0,14.0,32.0,9.0,17.0,0.0,0.0,0.0,4.0,83.3,14.3,0.0,0.0,179407.0,15030.0,13000.0,11325.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,

In [4]:
loans.columns.to_list()

['Unnamed: 0',
 'id',
 'member_id',
 'loan_amnt',
 'funded_amnt',
 'funded_amnt_inv',
 'term',
 'int_rate',
 'installment',
 'grade',
 'sub_grade',
 'emp_title',
 'emp_length',
 'home_ownership',
 'annual_inc',
 'verification_status',
 'issue_d',
 'loan_status',
 'pymnt_plan',
 'desc',
 'purpose',
 'title',
 'zip_code',
 'addr_state',
 'dti',
 'delinq_2yrs',
 'earliest_cr_line',
 'fico_range_low',
 'fico_range_high',
 'inq_last_6mths',
 'mths_since_last_delinq',
 'mths_since_last_record',
 'open_acc',
 'pub_rec',
 'revol_bal',
 'revol_util',
 'total_acc',
 'initial_list_status',
 'out_prncp',
 'out_prncp_inv',
 'total_pymnt',
 'total_pymnt_inv',
 'total_rec_prncp',
 'total_rec_int',
 'total_rec_late_fee',
 'recoveries',
 'collection_recovery_fee',
 'last_pymnt_d',
 'last_pymnt_amnt',
 'next_pymnt_d',
 'last_credit_pull_d',
 'last_fico_range_high',
 'last_fico_range_low',
 'collections_12_mths_ex_med',
 'mths_since_last_major_derog',
 'policy_code',
 'application_type',
 'annual_inc_joi

## Tabla (column_name, type)

Revisa el metodo pd.DataFrame.dtypes. https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.dtypes.html 

In [5]:
column_types = loans.dtypes
column_types.to_list()

[dtype('int64'),
 dtype('int64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('O'),
 dtype('float64'),
 dtype('float64'),
 dtype('O'),
 dtype('O'),
 dtype('O'),
 dtype('O'),
 dtype('O'),
 dtype('float64'),
 dtype('O'),
 dtype('O'),
 dtype('O'),
 dtype('O'),
 dtype('O'),
 dtype('O'),
 dtype('O'),
 dtype('O'),
 dtype('O'),
 dtype('float64'),
 dtype('float64'),
 dtype('O'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('O'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('O'),
 dtype('float64'),
 dtype('O'),
 dtype('O'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('O'),
 dtype('float64'),
 dtype('float64'),
 dtype('f

In [6]:
loans.isna().sum()


Unnamed: 0                    0
id                            0
member_id                100000
loan_amnt                     0
funded_amnt                   0
                          ...  
settlement_status         98710
settlement_date           98710
settlement_amount         98710
settlement_percentage     98710
settlement_term           98710
Length: 151, dtype: int64

## Cargar descripcion de columnas

La siguiente tabla tiene una descripcion del significado de cada columna

In [7]:
datos_dict = pd.read_excel(
    'https://resources.lendingclub.com/LCDataDictionary.xlsx')
datos_dict.columns = ['feature', 'description']

In [8]:
datos_dict

,feature,description
0,acc_now_delinq,The number of accounts on which the borrower i...
1,acc_open_past_24mths,Number of trades opened in past 24 months.
2,addr_state,The state provided by the borrower in the loan...
3,all_util,Balance to credit limit on all trades
4,annual_inc,The self-reported annual income provided by th...
...,...,...
148,settlement_amount,The loan amount that the borrower has agreed t...
149,settlement_percentage,The settlement amount as a percentage of the p...
150,settlement_term,The number of months that the borrower will be...
151,NaN,NaN


### Pickle

Crea codigo para **guardar** y **cargar** el DataFrame de `datos_dict` creada en las celdas anteriores en formato **pickle**

In [9]:
# Codigo guardar
import pickle as pk

with open('datos_dict.pickle', 'wb') as f:
    pk.dump(datos_dict, f)


In [10]:
# Codigo para cargar
with open('datos_dict.pickle', 'rb') as f:
    columns_desc=pk.load(f)

columns_desc

,feature,description
0,acc_now_delinq,The number of accounts on which the borrower i...
1,acc_open_past_24mths,Number of trades opened in past 24 months.
2,addr_state,The state provided by the borrower in the loan...
3,all_util,Balance to credit limit on all trades
4,annual_inc,The self-reported annual income provided by th...
...,...,...
148,settlement_amount,The loan amount that the borrower has agreed t...
149,settlement_percentage,The settlement amount as a percentage of the p...
150,settlement_term,The number of months that the borrower will be...
151,NaN,NaN


In [11]:
#Eliminando todas las columnas cuyo contenido en un 95% o mas sea NaN y la columna inservible Unnamed: 0
loans_filtered = loans.dropna(thresh=len(loans)*.95, axis=1)
loans_filtered.drop('Unnamed: 0', axis=1, inplace=True)
loans_filtered

/tmp/ipykernel_4200/2180864715.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  loans_filtered.drop('Unnamed: 0', axis=1, inplace=True)


,id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,fico_range_low,fico_range_high,inq_last_6mths,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,last_credit_pull_d,last_fico_range_high,last_fico_range_low,collections_12_mths_ex_med,policy_code,application_type,acc_now_delinq,tot_coll_amt,tot_cur_bal,total_rev_hi_lim,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,disbursement_method,debt_settlement_flag
0,38098114,15000.0,15000.0,15000.0,60 months,12.39,336.64,C,C1,RENT,78000.0,Source Verified,Dec-2014,Fully Paid,n,debt_consolidation,Debt consolidation,235xx,VA,12.03,0.0,Aug-1994,750.0,754.0,0.0,6.0,0.0,138008.0,29.0,17.0,w,0.00,0.00,17392.370000,17392.37,15000.00,2392.37,0.0,0.00,0.0000,Jun-2016,12017.81,Nov-2017,684.0,680.0,0.0,1.0,Individual,0.0,0.0,149140.0,184500.0,5.0,29828.0,9525.0,4.7,0.0,0.0,103.0,244.0,1.0,1.0,0.0,47.0,0.0,1.0,4.0,1.0,2.0,8.0,5.0,9.0,4.0,6.0,0.0,0.0,0.0,4.0,100.0,0.0,0.0,0.0,196500.0,149140.0,10000.0,12000.0,N,Cash,N
1,36805548,10400.0,10400.0,10400.0,36 months,6.99,321.08,A,A3,MORTGAGE,58000.0,Not Verified,Dec-2014,Charged Off,n,credit_card,Credit card refinancing,937xx,CA,14.92,0.0,Sep-1989,710.0,714.0,2.0,17.0,0.0,6133.0,31.6,36.0,w,0.00,0.00,6611.690000,6611.69,5217.75,872.67,0.0,521.27,93.8286,Aug-2016,321.08,Feb-2017,564.0,560.0,0.0,1.0,Individual,0.0,0.0,162110.0,19400.0,7.0,9536.0,7599.0,41.5,0.0,0.0,76.0,290.0,1.0,1.0,1.0,5.0,4.0,6.0,9.0,7.0,18.0,2.0,14.0,32.0,9.0,17.0,0.0,0.0,0.0,4.0,83.3,14.3,0.0,0.0,179407.0,15030.0,13000.0,11325.0,N,Cash,N
2,37842129,21425.0,21425.0,21425.0,60 months,15.59,516.36,D,D1,RENT,63800.0,Source Verified,Dec-2014,Fully Paid,n,credit_card,Credit card refinancing,658xx,MO,18.49,0.0,Aug-2003,685.0,689.0,0.0,10.0,0.0,16374.0,76.2,35.0,w,0.00,0.00,25512.200000,25512.20,21425.00,4087.20,0.0,0.00,0.0000,May-2016,17813.19,Dec-2017,704.0,700.0,0.0,1.0,Individual,0.0,0.0,42315.0,21500.0,4.0,4232.0,324.0,97.8,0.0,0.0,135.0,136.0,7.0,7.0,0.0,7.0,1.0,3.0,4.0,3.0,12.0,16.0,5.0,18.0,4.0,10.0,0.0,0.0,0.0,2.0,91.4,100.0,0.0,0.0,57073.0,42315.0,15000.0,35573.0,N,Cash,N
3,37612354,12800.0,12800.0,12800.0,60 months,17.14,319.08,D,D4,MORTGAGE,125000.0,Verified,Dec-2014,Current,n,car,Car financing,953xx,CA,8.31,1.0,Oct-2000,665.0,669.0,0.0,8.0,0.0,5753.0,100.9,13.0,w,6673.54,6673.54,11207.670000,11207.67,6126.46,5081.21,0.0,0.00,0.0000,Dec-2017,319.08,Dec-2017,629.0,625.0,0.0,1.0,Individual,0.0,0.0,261815.0,5700.0,2.0,32727.0,0.0,103.2,0.0,0.0,16.0,170.0,21.0,16.0,5.0,21.0,1.0,3.0,5.0,3.0,5.0,1.0,5.0,7.0,5.0,8.0,0.0,0.0,0.0,0.0,76.9,100.0,0.0,0.0,368700.0,18007.0,4400.0,18000.0,N,Cash,N
4,37662224,7650.0,7650.0,7650.0,36 months,13.66,260.20,C,C3,RENT,50000.0,Source Verified,Dec-2014,Charged Off,n,debt_consolidation,Debt consolidation,850xx,AZ,34.81,0.0,Aug-2002,685.0,689.0,1.0,11.0,0.0,16822.0,91.9,20.0,f,0.00,0.00,2281.980000,2281.98,704.38,339.61,0.0,1237.99,222.8382,Aug-2015,17.70,Oct-2016,559.0,555.0,0.0,1.0,Individual,0.0,0.0,64426.0,18300.0,6.0,5857.0,332.0,93.2,0.0,0.0,137.0,148.0,8.0,8.0,0.0,17.0,0.0,1.0,4.0,1.0,4.0,12.0,4.0,8.0,4.0,11.0,0.0,0.0,0.0,2.0,100.0,100.0,0.0,0.0,82331.0,64426.0,4900.0,64031.0,N,Cash,N
...

In [12]:
loans_filtered.dtypes


id                              int64
loan_amnt                     float64
funded_amnt                   float64
funded_amnt_inv               float64
term                           object
                               ...   
total_bc_limit                float64
total_il_high_credit_limit    float64
hardship_flag                  object
disbursement_method            object
debt_settlement_flag           object
Length: 89, dtype: object

## Tipos de Datos

Realiza las transformaciones o casteos (casting) que creas necesarios a tus datos de tal manera que el tipo de dato sea adecuado. Al terminar recrea la tabla `column_types` con los nuevos tipos.

No olvides anotar tus justificaciones para recordar cuando te toque explicarlo.

In [13]:

# Manejos de tipos 1
# Tu codigo aqui
loans_filtered[['issue_d', 'earliest_cr_line', 'last_pymnt_d', 'last_credit_pull_d']]=loans_filtered[['issue_d', 'earliest_cr_line', 'last_pymnt_d', 'last_credit_pull_d']].apply(pd.to_datetime,format="%b-%Y")




/tmp/ipykernel_4200/1304959315.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  loans_filtered[['issue_d', 'earliest_cr_line', 'last_pymnt_d', 'last_credit_pull_d']]=loans_filtered[['issue_d', 'earliest_cr_line', 'last_pymnt_d', 'last_credit_pull_d']].apply(pd.to_datetime,format="%b-%Y")


In [14]:
column_types =loans_filtered.dtypes
column_types

id                              int64
loan_amnt                     float64
funded_amnt                   float64
funded_amnt_inv               float64
term                           object
                               ...   
total_bc_limit                float64
total_il_high_credit_limit    float64
hardship_flag                  object
disbursement_method            object
debt_settlement_flag           object
Length: 89, dtype: object

## Manejo de NaNs o missings

Maneja los datos de tipos missing. Elije una estrategia adecuada dependiendo del tipo de dato que le asignaste a la columna.


In [15]:
nans=loans_filtered.isna().sum()  #vemos que aun existen NaN's en mis datos
types=loans_filtered.dtypes

In [16]:
#DataFrame que muestra la columna, el tipo de datos y cuantos nans contiene
v1=pd.concat([nans,types],axis=1)
v1=v1.rename(columns={0: 'NaCount', 1:'type'})
faltantes=v1.loc[v1['NaCount']>0]
faltantes

,NaCount,type
revol_util,56,float64
last_pymnt_d,67,datetime64[ns]
last_credit_pull_d,17,datetime64[ns]
bc_open_to_buy,1135,float64
bc_util,1198,float64
mo_sin_old_il_acct,3007,float64
mths_since_recent_bc,1049,float64
num_tl_120dpd_2m,1956,float64
percent_bc_gt_75,1161,float64


In [17]:
#Sustituimos los NaN en NaT
loans_complete=loans_filtered.fillna('missing')
types=loans_complete.dtypes
nans=loans_complete.isna().sum()

In [18]:
v2=pd.concat([nans, types], axis=1)
v2=v2.rename(columns={0:'NaN_Count', 1:'type'})
faltantes=v2.loc[v2['NaN_Count']>0]
faltantes

,NaN_Count,type


In [19]:
loans_complete


,id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,fico_range_low,fico_range_high,inq_last_6mths,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,last_credit_pull_d,last_fico_range_high,last_fico_range_low,collections_12_mths_ex_med,policy_code,application_type,acc_now_delinq,tot_coll_amt,tot_cur_bal,total_rev_hi_lim,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,disbursement_method,debt_settlement_flag
0,38098114,15000.0,15000.0,15000.0,60 months,12.39,336.64,C,C1,RENT,78000.0,Source Verified,2014-12-01,Fully Paid,n,debt_consolidation,Debt consolidation,235xx,VA,12.03,0.0,1994-08-01,750.0,754.0,0.0,6.0,0.0,138008.0,29.0,17.0,w,0.00,0.00,17392.370000,17392.37,15000.00,2392.37,0.0,0.00,0.0000,2016-06-01 00:00:00,12017.81,2017-11-01 00:00:00,684.0,680.0,0.0,1.0,Individual,0.0,0.0,149140.0,184500.0,5.0,29828.0,9525.0,4.7,0.0,0.0,103.0,244.0,1.0,1.0,0.0,47.0,0.0,1.0,4.0,1.0,2.0,8.0,5.0,9.0,4.0,6.0,0.0,0.0,0.0,4.0,100.0,0.0,0.0,0.0,196500.0,149140.0,10000.0,12000.0,N,Cash,N
1,36805548,10400.0,10400.0,10400.0,36 months,6.99,321.08,A,A3,MORTGAGE,58000.0,Not Verified,2014-12-01,Charged Off,n,credit_card,Credit card refinancing,937xx,CA,14.92,0.0,1989-09-01,710.0,714.0,2.0,17.0,0.0,6133.0,31.6,36.0,w,0.00,0.00,6611.690000,6611.69,5217.75,872.67,0.0,521.27,93.8286,2016-08-01 00:00:00,321.08,2017-02-01 00:00:00,564.0,560.0,0.0,1.0,Individual,0.0,0.0,162110.0,19400.0,7.0,9536.0,7599.0,41.5,0.0,0.0,76.0,290.0,1.0,1.0,1.0,5.0,4.0,6.0,9.0,7.0,18.0,2.0,14.0,32.0,9.0,17.0,0.0,0.0,0.0,4.0,83.3,14.3,0.0,0.0,179407.0,15030.0,13000.0,11325.0,N,Cash,N
2,37842129,21425.0,21425.0,21425.0,60 months,15.59,516.36,D,D1,RENT,63800.0,Source Verified,2014-12-01,Fully Paid,n,credit_card,Credit card refinancing,658xx,MO,18.49,0.0,2003-08-01,685.0,689.0,0.0,10.0,0.0,16374.0,76.2,35.0,w,0.00,0.00,25512.200000,25512.20,21425.00,4087.20,0.0,0.00,0.0000,2016-05-01 00:00:00,17813.19,2017-12-01 00:00:00,704.0,700.0,0.0,1.0,Individual,0.0,0.0,42315.0,21500.0,4.0,4232.0,324.0,97.8,0.0,0.0,135.0,136.0,7.0,7.0,0.0,7.0,1.0,3.0,4.0,3.0,12.0,16.0,5.0,18.0,4.0,10.0,0.0,0.0,0.0,2.0,91.4,100.0,0.0,0.0,57073.0,42315.0,15000.0,35573.0,N,Cash,N
3,37612354,12800.0,12800.0,12800.0,60 months,17.14,319.08,D,D4,MORTGAGE,125000.0,Verified,2014-12-01,Current,n,car,Car financing,953xx,CA,8.31,1.0,2000-10-01,665.0,669.0,0.0,8.0,0.0,5753.0,100.9,13.0,w,6673.54,6673.54,11207.670000,11207.67,6126.46,5081.21,0.0,0.00,0.0000,2017-12-01 00:00:00,319.08,2017-12-01 00:00:00,629.0,625.0,0.0,1.0,Individual,0.0,0.0,261815.0,5700.0,2.0,32727.0,0.0,103.2,0.0,0.0,16.0,170.0,21.0,16.0,5.0,21.0,1.0,3.0,5.0,3.0,5.0,1.0,5.0,7.0,5.0,8.0,0.0,0.0,0.0,0.0,76.9,100.0,0.0,0.0,368700.0,18007.0,4400.0,18000.0,N,Cash,N
4,37662224,7650.0,7650.0,7650.0,36 months,13.66,260.20,C,C3,RENT,50000.0,Source Verified,2014-12-01,Charged Off,n,debt_consolidation,Debt consolidation,850xx,AZ,34.81,0.0,2002-08-01,685.0,689.0,1.0,11.0,0.0,16822.0,91.9,20.0,f,0.00,0.00,2281.980000,2281.98,704.38,339.61,0.0,1237.99,222.8382,2015-08-01 00:00:00,17.70,2016-10-01 00:00:00,559.0,555.0,0.0,1.0,Individual,0.0,0.0,64426.0,18300.0,6.0,5857.0,332.0,93.2,0.0,0.0,137.0,148.0,8.0,8.0,

Crea codigo para **guardar** y **cargar** un archivo JSON en el que se guarde la `estrategia` y `valor` que utilizaste para **imputar**. Por ejemplo: Si hay una columna que se llama `columna 3` y utilizaste la estrategia de imputacion de media, y existe otra llamada `columna 4` y  elegiste la palabra 'missing' el JSON debera contener:  
  
 `{'columna 3':{'estrategia':'mean', 'valor':3.4}, 'columna 4':{'estrategia':'identificador', 'valor':'missing'}}`  

 De tal manera que para cada columna que tenga un metodo de imputacion apunte a otro diccionario donde el **key** `estrategia` describa de manera sencilla el metodo, y el **key** `valor` el valor usado. En general:   
 `{'nombre de la columna':{'estrategia':'descripcion de estrategia', 'valor':'valor utilizado'}}`. 
 

De utilizar mas de un metodo puedes anidarlos en una lista  
  `[{...},{...}]`.  

Incluso si la columna utilizada no sufrio imputacion, es necesario que la agregues al JSON.

La idea es que cualquier otra persona pueda cargar el el archivo JSON con tu funcion, entender que hiciste y replicarlo facilmente. No existe solo una respuesta correcta, pero tendras que justificar y explicar tus deciciones.

In [20]:
columns_lim=loans_complete[['id', 'loan_amnt', 'funded_amnt', 'funded_amnt_inv', 'term', 'int_rate',
       'installment', 'grade', 'sub_grade', 'home_ownership', 'annual_inc',
       'verification_status', 'issue_d', 'loan_status', 'pymnt_plan',
       'purpose', 'title', 'zip_code', 'addr_state', 'dti', 'delinq_2yrs',
       'earliest_cr_line', 'fico_range_low', 'fico_range_high',
       'inq_last_6mths', 'open_acc', 'pub_rec', 'revol_bal', 'revol_util',
       'total_acc', 'initial_list_status', 'out_prncp', 'out_prncp_inv',
       'total_pymnt', 'total_pymnt_inv', 'total_rec_prncp', 'total_rec_int',
       'total_rec_late_fee', 'recoveries', 'collection_recovery_fee',
       'last_pymnt_d', 'last_pymnt_amnt', 'last_credit_pull_d',
       'last_fico_range_high', 'last_fico_range_low',
       'collections_12_mths_ex_med', 'policy_code', 'application_type',
       'acc_now_delinq', 'tot_coll_amt', 'tot_cur_bal', 'total_rev_hi_lim',
       'acc_open_past_24mths', 'avg_cur_bal', 'bc_open_to_buy', 'bc_util',
       'chargeoff_within_12_mths', 'delinq_amnt', 'mo_sin_old_il_acct',
       'mo_sin_old_rev_tl_op', 'mo_sin_rcnt_rev_tl_op', 'mo_sin_rcnt_tl',
       'mort_acc', 'mths_since_recent_bc', 'num_accts_ever_120_pd',
       'num_actv_bc_tl', 'num_actv_rev_tl', 'num_bc_sats', 'num_bc_tl',
       'num_il_tl', 'num_op_rev_tl', 'num_rev_accts', 'num_rev_tl_bal_gt_0',
       'num_sats', 'num_tl_120dpd_2m', 'num_tl_30dpd', 'num_tl_90g_dpd_24m',
       'num_tl_op_past_12m', 'pct_tl_nvr_dlq', 'percent_bc_gt_75',
       'pub_rec_bankruptcies', 'tax_liens',
       'total_bal_ex_mort',
       'hardship_flag', 'disbursement_method', 'debt_settlement_flag']]

### Imputacion

In [21]:
# Tu codigo aqui
#melt para que los tipos de limite de credito sean valores
#credit_limit=loans_filtered[['tot_hi_cred_lim', 'total_il_high_credit_limit', 'total_bc_limit']]
loans_limit=pd.melt(loans_complete, id_vars=columns_lim, value_vars=['tot_hi_cred_lim',
                                                             'total_il_high_credit_limit','total_bc_limit'],var_name='type_of_limit',
                                                               value_name='limit')
loans_limit.head()

,id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,fico_range_low,fico_range_high,inq_last_6mths,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,last_credit_pull_d,last_fico_range_high,last_fico_range_low,collections_12_mths_ex_med,policy_code,application_type,acc_now_delinq,tot_coll_amt,tot_cur_bal,total_rev_hi_lim,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,total_bal_ex_mort,hardship_flag,disbursement_method,debt_settlement_flag,type_of_limit,limit
0,38098114,15000.0,15000.0,15000.0,60 months,12.39,336.64,C,C1,RENT,78000.0,Source Verified,2014-12-01,Fully Paid,n,debt_consolidation,Debt consolidation,235xx,VA,12.03,0.0,1994-08-01,750.0,754.0,0.0,6.0,0.0,138008.0,29.0,17.0,w,0.00,0.00,17392.37,17392.37,15000.00,2392.37,0.0,0.00,0.0000,2016-06-01 00:00:00,12017.81,2017-11-01 00:00:00,684.0,680.0,0.0,1.0,Individual,0.0,0.0,149140.0,184500.0,5.0,29828.0,9525.0,4.7,0.0,0.0,103.0,244.0,1.0,1.0,0.0,47.0,0.0,1.0,4.0,1.0,2.0,8.0,5.0,9.0,4.0,6.0,0.0,0.0,0.0,4.0,100.0,0.0,0.0,0.0,149140.0,N,Cash,N,tot_hi_cred_lim,196500.0
1,36805548,10400.0,10400.0,10400.0,36 months,6.99,321.08,A,A3,MORTGAGE,58000.0,Not Verified,2014-12-01,Charged Off,n,credit_card,Credit card refinancing,937xx,CA,14.92,0.0,1989-09-01,710.0,714.0,2.0,17.0,0.0,6133.0,31.6,36.0,w,0.00,0.00,6611.69,6611.69,5217.75,872.67,0.0,521.27,93.8286,2016-08-01 00:00:00,321.08,2017-02-01 00:00:00,564.0,560.0,0.0,1.0,Individual,0.0,0.0,162110.0,19400.0,7.0,9536.0,7599.0,41.5,0.0,0.0,76.0,290.0,1.0,1.0,1.0,5.0,4.0,6.0,9.0,7.0,18.0,2.0,14.0,32.0,9.0,17.0,0.0,0.0,0.0,4.0,83.3,14.3,0.0,0.0,15030.0,N,Cash,N,tot_hi_cred_lim,179407.0
2,37842129,21425.0,21425.0,21425.0,60 months,15.59,516.36,D,D1,RENT,63800.0,Source Verified,2014-12-01,Fully Paid,n,credit_card,Credit card refinancing,658xx,MO,18.49,0.0,2003-08-01,685.0,689.0,0.0,10.0,0.0,16374.0,76.2,35.0,w,0.00,0.00,25512.20,25512.20,21425.00,4087.20,0.0,0.00,0.0000,2016-05-01 00:00:00,17813.19,2017-12-01 00:00:00,704.0,700.0,0.0,1.0,Individual,0.0,0.0,42315.0,21500.0,4.0,4232.0,324.0,97.8,0.0,0.0,135.0,136.0,7.0,7.0,0.0,7.0,1.0,3.0,4.0,3.0,12.0,16.0,5.0,18.0,4.0,10.0,0.0,0.0,0.0,2.0,91.4,100.0,0.0,0.0,42315.0,N,Cash,N,tot_hi_cred_lim,57073.0
3,37612354,12800.0,12800.0,12800.0,60 months,17.14,319.08,D,D4,MORTGAGE,125000.0,Verified,2014-12-01,Current,n,car,Car financing,953xx,CA,8.31,1.0,2000-10-01,665.0,669.0,0.0,8.0,0.0,5753.0,100.9,13.0,w,6673.54,6673.54,11207.67,11207.67,6126.46,5081.21,0.0,0.00,0.0000,2017-12-01 00:00:00,319.08,2017-12-01 00:00:00,629.0,625.0,0.0,1.0,Individual,0.0,0.0,261815.0,5700.0,2.0,32727.0,0.0,103.2,0.0,0.0,16.0,170.0,21.0,16.0,5.0,21.0,1.0,3.0,5.0,3.0,5.0,1.0,5.0,7.0,5.0,8.0,0.0,0.0,0.0,0.0,76.9,100.0,0.0,0.0,18007.0,N,Cash,N,tot_hi_cred_lim,368700.0
4,37662224,7650.0,7650.0,7650.0,36 months,13.66,260.20,C,C3,RENT,50000.0,Source Verified,2014-12-01,Charged Off,n,debt_consolidation,Debt consolidation,850xx,AZ,34.81,0.0,2002-08-01,685.0,689.0,1.0,11.0,0.0,16822.0,91.9,20.0,f,0.00,0.00,2281.98,2281.98,704.38,339.61,0.0,1237.99,222.8382,2015-08-01 00:00:00,17.70,2016-10-01 00:00:00,559.0,555.0,0.0,1.0,Individual,0.0,0.0,64426.0,18300.0,6.0,5857.0,332.0,93.2,0.0,0.0,137.0,148.0,8.0,8.0,0.0,17.0,0.0,1.0,4.0,1.0,4.0,12.0,4.0,8.0,4.0,11.0,0.0,0.

In [22]:
columns_amnt=loans_limit[['id', 'term', 'int_rate',
       'installment', 'grade', 'sub_grade', 'home_ownership', 'annual_inc',
       'verification_status', 'issue_d', 'loan_status', 'pymnt_plan',
       'purpose', 'title', 'zip_code', 'addr_state', 'dti', 'delinq_2yrs',
       'earliest_cr_line', 'fico_range_low', 'fico_range_high',
       'inq_last_6mths', 'open_acc', 'pub_rec', 'revol_bal', 'revol_util',
       'total_acc', 'initial_list_status', 'out_prncp', 'out_prncp_inv',
       'total_pymnt', 'total_pymnt_inv', 'total_rec_prncp', 'total_rec_int',
       'total_rec_late_fee', 'recoveries', 'collection_recovery_fee',
       'last_pymnt_d', 'last_pymnt_amnt', 'last_credit_pull_d',
       'last_fico_range_high', 'last_fico_range_low',
       'collections_12_mths_ex_med', 'policy_code', 'application_type',
       'acc_now_delinq', 'tot_coll_amt', 'tot_cur_bal', 'total_rev_hi_lim',
       'acc_open_past_24mths', 'avg_cur_bal', 'bc_open_to_buy', 'bc_util',
       'chargeoff_within_12_mths', 'delinq_amnt', 'mo_sin_old_il_acct',
       'mo_sin_old_rev_tl_op', 'mo_sin_rcnt_rev_tl_op', 'mo_sin_rcnt_tl',
       'mort_acc', 'mths_since_recent_bc', 'num_accts_ever_120_pd',
       'num_actv_bc_tl', 'num_actv_rev_tl', 'num_bc_sats', 'num_bc_tl',
       'num_il_tl', 'num_op_rev_tl', 'num_rev_accts', 'num_rev_tl_bal_gt_0',
       'num_sats', 'num_tl_120dpd_2m', 'num_tl_30dpd', 'num_tl_90g_dpd_24m',
       'num_tl_op_past_12m', 'pct_tl_nvr_dlq', 'percent_bc_gt_75',
       'pub_rec_bankruptcies', 'tax_liens', 'total_bal_ex_mort',
       'hardship_flag', 'disbursement_method', 'debt_settlement_flag',
       'type_of_limit', 'limit']]
columns_amnt

,id,term,int_rate,installment,grade,sub_grade,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,fico_range_low,fico_range_high,inq_last_6mths,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,last_credit_pull_d,last_fico_range_high,last_fico_range_low,collections_12_mths_ex_med,policy_code,application_type,acc_now_delinq,tot_coll_amt,tot_cur_bal,total_rev_hi_lim,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,total_bal_ex_mort,hardship_flag,disbursement_method,debt_settlement_flag,type_of_limit,limit
0,38098114,60 months,12.39,336.64,C,C1,RENT,78000.0,Source Verified,2014-12-01,Fully Paid,n,debt_consolidation,Debt consolidation,235xx,VA,12.03,0.0,1994-08-01,750.0,754.0,0.0,6.0,0.0,138008.0,29.0,17.0,w,0.00,0.00,17392.370000,17392.37,15000.00,2392.37,0.0,0.00,0.0000,2016-06-01 00:00:00,12017.81,2017-11-01 00:00:00,684.0,680.0,0.0,1.0,Individual,0.0,0.0,149140.0,184500.0,5.0,29828.0,9525.0,4.7,0.0,0.0,103.0,244.0,1.0,1.0,0.0,47.0,0.0,1.0,4.0,1.0,2.0,8.0,5.0,9.0,4.0,6.0,0.0,0.0,0.0,4.0,100.0,0.0,0.0,0.0,149140.0,N,Cash,N,tot_hi_cred_lim,196500.0
1,36805548,36 months,6.99,321.08,A,A3,MORTGAGE,58000.0,Not Verified,2014-12-01,Charged Off,n,credit_card,Credit card refinancing,937xx,CA,14.92,0.0,1989-09-01,710.0,714.0,2.0,17.0,0.0,6133.0,31.6,36.0,w,0.00,0.00,6611.690000,6611.69,5217.75,872.67,0.0,521.27,93.8286,2016-08-01 00:00:00,321.08,2017-02-01 00:00:00,564.0,560.0,0.0,1.0,Individual,0.0,0.0,162110.0,19400.0,7.0,9536.0,7599.0,41.5,0.0,0.0,76.0,290.0,1.0,1.0,1.0,5.0,4.0,6.0,9.0,7.0,18.0,2.0,14.0,32.0,9.0,17.0,0.0,0.0,0.0,4.0,83.3,14.3,0.0,0.0,15030.0,N,Cash,N,tot_hi_cred_lim,179407.0
2,37842129,60 months,15.59,516.36,D,D1,RENT,63800.0,Source Verified,2014-12-01,Fully Paid,n,credit_card,Credit card refinancing,658xx,MO,18.49,0.0,2003-08-01,685.0,689.0,0.0,10.0,0.0,16374.0,76.2,35.0,w,0.00,0.00,25512.200000,25512.20,21425.00,4087.20,0.0,0.00,0.0000,2016-05-01 00:00:00,17813.19,2017-12-01 00:00:00,704.0,700.0,0.0,1.0,Individual,0.0,0.0,42315.0,21500.0,4.0,4232.0,324.0,97.8,0.0,0.0,135.0,136.0,7.0,7.0,0.0,7.0,1.0,3.0,4.0,3.0,12.0,16.0,5.0,18.0,4.0,10.0,0.0,0.0,0.0,2.0,91.4,100.0,0.0,0.0,42315.0,N,Cash,N,tot_hi_cred_lim,57073.0
3,37612354,60 months,17.14,319.08,D,D4,MORTGAGE,125000.0,Verified,2014-12-01,Current,n,car,Car financing,953xx,CA,8.31,1.0,2000-10-01,665.0,669.0,0.0,8.0,0.0,5753.0,100.9,13.0,w,6673.54,6673.54,11207.670000,11207.67,6126.46,5081.21,0.0,0.00,0.0000,2017-12-01 00:00:00,319.08,2017-12-01 00:00:00,629.0,625.0,0.0,1.0,Individual,0.0,0.0,261815.0,5700.0,2.0,32727.0,0.0,103.2,0.0,0.0,16.0,170.0,21.0,16.0,5.0,21.0,1.0,3.0,5.0,3.0,5.0,1.0,5.0,7.0,5.0,8.0,0.0,0.0,0.0,0.0,76.9,100.0,0.0,0.0,18007.0,N,Cash,N,tot_hi_cred_lim,368700.0
4,37662224,36 months,13.66,260.20,C,C3,RENT,50000.0,Source Verified,2014-12-01,Charged Off,n,debt_consolidation,Debt consolidation,850xx,AZ,34.81,0.0,2002-08-01,685.0,689.0,1.0,11.0,0.0,16822.0,91.9,20.0,f,0.00,0.00,2281.980000,2281.98,704.38,339.61,0.0,1237.99,222.8382,2015-08-01 00:00:00,17.70,2016-10-01 00:00:00,559.0,555.0,0.0,1.0,Individual,0.0,0.0,64426.0,18300.0,6.0,5857.0,332.0,93.2,0.0,0.0,137.0,148.0,8.0,8.0,0.0,17.0,0.0,1.0,4.0,1.0,4.0,12.0,4.0,8.0,4.0,11.0,0.0,0.0,0.0,2.0,100.0,100.0,0.0,0.0,64426.0,N,Cash,N,tot_hi_cred_lim,82331.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,

In [23]:
#melt pero para los tipos de cantidades 
loans_amnt=pd.melt(loans_limit, id_vars=columns_amnt, 
                   value_vars=['loan_amnt','funded_amnt','funded_amnt_inv'], var_name='type_of_amnt', value_name='amnt')
loans_amnt.head()

,id,term,int_rate,installment,grade,sub_grade,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,fico_range_low,fico_range_high,inq_last_6mths,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,last_credit_pull_d,last_fico_range_high,last_fico_range_low,collections_12_mths_ex_med,policy_code,application_type,acc_now_delinq,tot_coll_amt,tot_cur_bal,total_rev_hi_lim,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,total_bal_ex_mort,hardship_flag,disbursement_method,debt_settlement_flag,type_of_limit,limit,type_of_amnt,amnt
0,38098114,60 months,12.39,336.64,C,C1,RENT,78000.0,Source Verified,2014-12-01,Fully Paid,n,debt_consolidation,Debt consolidation,235xx,VA,12.03,0.0,1994-08-01,750.0,754.0,0.0,6.0,0.0,138008.0,29.0,17.0,w,0.00,0.00,17392.37,17392.37,15000.00,2392.37,0.0,0.00,0.0000,2016-06-01 00:00:00,12017.81,2017-11-01 00:00:00,684.0,680.0,0.0,1.0,Individual,0.0,0.0,149140.0,184500.0,5.0,29828.0,9525.0,4.7,0.0,0.0,103.0,244.0,1.0,1.0,0.0,47.0,0.0,1.0,4.0,1.0,2.0,8.0,5.0,9.0,4.0,6.0,0.0,0.0,0.0,4.0,100.0,0.0,0.0,0.0,149140.0,N,Cash,N,tot_hi_cred_lim,196500.0,loan_amnt,15000.0
1,36805548,36 months,6.99,321.08,A,A3,MORTGAGE,58000.0,Not Verified,2014-12-01,Charged Off,n,credit_card,Credit card refinancing,937xx,CA,14.92,0.0,1989-09-01,710.0,714.0,2.0,17.0,0.0,6133.0,31.6,36.0,w,0.00,0.00,6611.69,6611.69,5217.75,872.67,0.0,521.27,93.8286,2016-08-01 00:00:00,321.08,2017-02-01 00:00:00,564.0,560.0,0.0,1.0,Individual,0.0,0.0,162110.0,19400.0,7.0,9536.0,7599.0,41.5,0.0,0.0,76.0,290.0,1.0,1.0,1.0,5.0,4.0,6.0,9.0,7.0,18.0,2.0,14.0,32.0,9.0,17.0,0.0,0.0,0.0,4.0,83.3,14.3,0.0,0.0,15030.0,N,Cash,N,tot_hi_cred_lim,179407.0,loan_amnt,10400.0
2,37842129,60 months,15.59,516.36,D,D1,RENT,63800.0,Source Verified,2014-12-01,Fully Paid,n,credit_card,Credit card refinancing,658xx,MO,18.49,0.0,2003-08-01,685.0,689.0,0.0,10.0,0.0,16374.0,76.2,35.0,w,0.00,0.00,25512.20,25512.20,21425.00,4087.20,0.0,0.00,0.0000,2016-05-01 00:00:00,17813.19,2017-12-01 00:00:00,704.0,700.0,0.0,1.0,Individual,0.0,0.0,42315.0,21500.0,4.0,4232.0,324.0,97.8,0.0,0.0,135.0,136.0,7.0,7.0,0.0,7.0,1.0,3.0,4.0,3.0,12.0,16.0,5.0,18.0,4.0,10.0,0.0,0.0,0.0,2.0,91.4,100.0,0.0,0.0,42315.0,N,Cash,N,tot_hi_cred_lim,57073.0,loan_amnt,21425.0
3,37612354,60 months,17.14,319.08,D,D4,MORTGAGE,125000.0,Verified,2014-12-01,Current,n,car,Car financing,953xx,CA,8.31,1.0,2000-10-01,665.0,669.0,0.0,8.0,0.0,5753.0,100.9,13.0,w,6673.54,6673.54,11207.67,11207.67,6126.46,5081.21,0.0,0.00,0.0000,2017-12-01 00:00:00,319.08,2017-12-01 00:00:00,629.0,625.0,0.0,1.0,Individual,0.0,0.0,261815.0,5700.0,2.0,32727.0,0.0,103.2,0.0,0.0,16.0,170.0,21.0,16.0,5.0,21.0,1.0,3.0,5.0,3.0,5.0,1.0,5.0,7.0,5.0,8.0,0.0,0.0,0.0,0.0,76.9,100.0,0.0,0.0,18007.0,N,Cash,N,tot_hi_cred_lim,368700.0,loan_amnt,12800.0
4,37662224,36 months,13.66,260.20,C,C3,RENT,50000.0,Source Verified,2014-12-01,Charged Off,n,debt_consolidation,Debt consolidation,850xx,AZ,34.81,0.0,2002-08-01,685.0,689.0,1.0,11.0,0.0,16822.0,91.9,20.0,f,0.00,0.00,2281.98,2281.98,704.38,339.61,0.0,1237.99,222.8382,2015-08-01 00:00:00,17.70,2016-10-01 00:00:00,559.0,555.0,0.0,1.0,Individual,0.0,0.0,64426.0,18300.0,6.0,5857.0,332.0,93.2,0.0,0.0,137.0,148.0,8.0,8.0,0.0,17.0,0.0,1.0,4.0,1.0,4.0,12.0,4.0,8.0,4.0,11.0,0.0,0.0,0.0,2.0,100.0,100.0,0.0,0.0,64426.0,N,Cash,N,tot_hi_cred_lim,82

### Codigo para salvar y cargar JSONs

In [24]:
#Primero creo una lista para los valores de las columnas para convertirlas en indexes.
keys=list(loans_amnt.columns)
#Luego, una variable que guarde cuantos 'missing' hay en cada columna.
vals=[None]*len(keys)
#Creamos una tupla con las columnas y los valores, empleando la funcion zip.
valor_llave=zip(keys,vals)
#Creamos un diccionario con la tupla.
dic={k:v for k,v in valor_llave}
#Creamos una lista con los valores de las columnas que contienen 'missing'.
list_missing=['emp_title', 'emp_length', 'title', 'revol_util', 'last_pymnt_d', 'last_credit_pull_d', 
              'collections_12_mths_ex_med', 'tot_coll_amt', 'tot_cur_bal', 'total_rev_hi_lim', 
              'avg_cur_bal', 'bc_open_to_buy', 'bc_util', 'mo_sin_old_il_acct', 'mo_sin_old_rev_tl_op', 
              'mo_sin_rcnt_rev_tl_op', 'mo_sin_rcnt_tl', 'mths_since_recent_bc', 'num_tl_120dpd_2m', 'pct_tl_nvr_dlq', 'percent_bc_gt_75']
#Creamos una lista con los las columnas que fueron eliminadas.
sum_nans=loans.isna().sum()
empty_cols=loans.loc[:, sum_nans>=len(loans)*.95]
list_empty=list(empty_cols.columns)


for col in list_missing:
    dic[col]={'estrategia':'Identificador', 'valor':'missing'}

for col in list_empty:
    dic[col]={'estrategia':'Eliminación', 'valor':'Ninguno'}


In [ ]:
#Creación del JSON
with(open('imputacion.json', 'w')) as file:
    json.dump(dic, file)

with(open('imputacion.json', 'r')) as file:
    imputaciones=json.load(file)